In [1331]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats
from dash import Dash, html, dcc, Input, Output
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score



1.	wstępna analiza danych i czyszczenie:
a.	duplikaty
b.	wartości odstające
c.	spójność
d.	braki w danych
e.	skala wartości
f.	inne
2.	wizualizacja rozkładu zmiennych, zależności ceny od innych zmiennych, liczebność kategorii
3.	budowa modelu regresji ceny od pozostałych zmiennych. Istotne zmienne należy wybrać eliminacją wsteczną lub selekcją postępującą.
4.	wizualizacja modelu regresji
5.	stworzenie dashboardu z powyższymi wizualizacjami i próbką danych w postaci tabeli. Dashboard powinien umożliwiać zmianę parametrów wykresów - np. zależność ceny od innej, wybranej przez użytkownika zmiennej

Wszystkie wizualizacje i dane powinny zostać zaprezentowane w postaci dashboardu. Etapy czyszczenia danych i budowy modelu regresji należy zawrzeć w notatniku Jupytera (.ipynb).


In [1332]:
#porządkowanie tabeli
df = pd.read_csv("messy_data.csv", names=['carat', 'clarity' , 'color' ,'cut', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price'])
df.drop(index=0, inplace=True)

In [1333]:
#odrzucenie duplikatów
df.drop_duplicates(subset=None, inplace=True)

In [1334]:
#zamiana dziwnych i pustych wartości na NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [1335]:
#konwersja na typ float
df['carat'] = df['carat'].astype(float)
df['x dimension'] = df['x dimension'].astype(float)
df['y dimension'] = df['y dimension'].astype(float)
df['z dimension'] = df['z dimension'].astype(float)
df['depth'] = df['depth'].astype(float)
df['table'] = df['table'].astype(float)
df['price'] = df['price'].astype(float)

#zamiana na duże litery
df['cut'] = df['cut'].str.upper()
df['clarity'] = df['clarity'].str.upper()
df['color'] = df['color'].str.upper()

In [1336]:
#zamiana NaN na średnią tam gdzie się da
df['carat'].fillna(df["carat"].mean(), inplace=True)
df['x dimension'].fillna(df["x dimension"].mean(), inplace=True)
df['y dimension'].fillna(df["y dimension"].mean(), inplace=True)
df['z dimension'].fillna(df["z dimension"].mean(), inplace=True)
df['depth'].fillna(df["depth"].mean(), inplace=True)
df['table'].fillna(df["table"].mean(), inplace=True)
df['price'].fillna(df["price"].mean(), inplace=True)

#i usunięcie tam gdzie się nie da
df.dropna(inplace=True)

In [1337]:
# Usuwanie wartości odstających
numeric_columns = ['carat', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price']

z = np.abs(stats.zscore(df['carat']))

threshold_z = 2
outlier_indices = np.where(z > threshold_z)[0]
no_outliers = df.drop(outlier_indices)
df = no_outliers

In [1338]:
colors = {
    'background' :'#ADD8E6',
    'text': '7FDBFF'
}

In [1339]:
app = Dash(__name__)
fig = px.scatter(df, x='carat', y='price', trendline='ols')


app.layout = html.Div(style={'backgroundColor': colors['background']},
                      children=[

                          html.H1(
                              children="Projekt PAD",
                              style={
                                  'text align': 'center',
                                  'color' : colors['text']
                              }
                          ),
                          html.Div(children="Interaktywny dashboard",
                                   style={
                                  'text align': 'center',
                                  'color' : colors['text']
                              }),
                        dcc.Dropdown(['carat', 'clarity' , 'color' ,'cut', 'x dimension',
                                                     'y dimension', 'z dimension', 'depth', 'table', 'price'],'carat', id='xdropdown'),
                        dcc.Dropdown(['carat', 'clarity', 'color' , 'cut', 'x dimension',
                                                     'y dimension', 'z dimension', 'depth', 'table', 'price'],'price', id='ydropdown'),
                                      
                                            
            dcc.Graph(
                id='graph',
                figure=fig
            )])

@app.callback(
    Output('graph', 'figure'),
    Input('xdropdown', 'value'),
    Input('ydropdown', 'value'))
def update_graph(x_column, y_column):
    fig = px.scatter(df, x=x_column, y=y_column, trendline='ols')
    fig.update_xaxes(type='category')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

In [1340]:
df

,carat,clarity,color,cut,x dimension,y dimension,z dimension,depth,table,price
1,0.500,IF,D,IDEAL,5.100000,5.150000,3.200000,61.500000,56.91018,3000.0
2,0.700,VVS2,E,PREMIUM,5.700000,5.829706,3.520000,62.000000,59.00000,4500.0
3,0.799,SI2,H,GOOD,4.300000,4.310000,3.694671,62.300000,56.00000,700.0
4,1.200,IF,D,IDEAL,5.885787,6.820000,4.200000,61.700000,58.00000,10000.0
5,0.900,I1,J,FAIR,6.000000,5.829706,3.700000,61.700000,56.91018,2400.0
...,...,...,...,...,...,...,...,...,...,...
196,0.450,SI1,F,VERY GOOD,4.900000,4.890000,3.694671,62.400000,59.00000,2300.0
197,1.250,VVS2,H,GOOD,6.900000,5.829706,4.270000,62.316541,55.00000,10400.0
198,0.550,I1,I,PREMIUM,5.300000,5.829706,3.280000,62.300000,57.00000,150000.0
199,0.850,SI2,E,FAIR,6.100000,6.130000,3.780000,62.316541,58.00000,6300.0


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File c:\Python311\Lib\site-packages\plotly\express\_core.py:335, in make_trace_kwargs(
    args={'animation_frame': None, 'animation_group': None, 'category_orders': {}, 'color': None, 'color_continuous_midpoint': None, 'color_continuous_scale': ['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921'], 'color_discrete_map': {}, 'color_discrete_sequence': ('#636efa', '#EF553B', '#00cc96', '#ab63fa', '#FFA15A', '#19d3f3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'), 'custom_data': None, 'data_frame':     color     price
0       D    3000.0
1       ...300.0
191     G    7500.0

[192 rows x 2 columns], ...},
    trace_spec=TraceSpec(constructor=<class 'plotly.graph_objs....'], trace_patch={'mode': 'lines'}, marginal=None),
    trace_data=    color     price
0       D    3000.0
1       ..